## Install Requirements

In [2]:
!pip -q install langchain
!pip -q install bitsandbytes accelerate transformers
!pip -q install datasets loralib sentencepiece
!pip -q install pypdf
!pip -q install sentence_transformers
!pip -q install unstructured # to extract unstrucutured data
!pip install tokenizers
!pip install faiss-cpu # As a vector database
!pip install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 1.2 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 MB 301.6 kB/s eta 0:00:00m eta 0:00:010:00:18
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 980.4 kB/s eta 0:00:00m eta 0:00:010:00:29
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 1.2 MB/s eta 0:00:00m eta 0:00:010:00:05
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 653.9 kB/s eta 0:00:00m eta 0:00:010:00:05
  Attempting uninstall: triton
    Found existing installation: triton 2.2.0
    Uninstalling triton-2.2.0:
      Successfully uninstalled triton-2.2.0
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.19.3
    Uninstalling nvidia-nccl-cu12-2.19.3:
      Successfully uninstalled nvidia-nccl-cu12-2.19.3
  Attempting uninstall: torch
    Found existing installation: torch 2.2.2
    Uninstalling torch-2.2.2:
      Successfully uninstalled torch-2.2.2


### Import Libraries

In [3]:
from langchain.document_loaders import UnstructuredURLLoader # Load data from urls
from langchain.text_splitter import CharacterTextSplitter # Split data into chunks
from langchain.embeddings import OpenAIEmbeddings # Create embeddings with the help of OpenAI API
from langchain.chat_models import ChatOpenAI # Use OpenAI as a chat model
from langchain.vectorstores import FAISS # Knowledge base
from langchain.vectorstores import Pinecone # Knowledge base
import pinecone
from langchain.chains import RetrievalQAWithSourcesChain # Provide answer with the references
from langchain.embeddings import HuggingFaceEmbeddings # Create embeddings with the help of Hugging Face
from transformers import AutoTokenizer, AutoModelForCausalLM # Create embeddings with the help of Hugging Face
from transformers import pipeline 
from langchain import HuggingFacePipeline
from huggingface_hub import notebook_login
import textwrap
import sys
import os
import torch
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

/home/shihab/learning_projects/custom-chat-bot-using-LLM/chatbotenv/lib/python3.10/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
[nltk_data] Downloading package punkt to /home/shihab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/shihab/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## URLS

In [4]:

URLs=[
    'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb',
    'https://www.mosaicml.com/blog/mpt-7b',
    'https://stability.ai/blog/stability-ai-launches-the-first-of-its-stablelm-suite-of-language-models',
    'https://lmsys.org/blog/2023-03-30-vicuna/'
]

In [5]:
loaders=UnstructuredURLLoader(urls=URLs)
data=loaders.load()

In [6]:
data

[Document(page_content='Open in app\n\nSign up\n\nSign in\n\nWrite\n\nSign up\n\nSign in\n\nPaper Review\n\nPaper Review: Llama 2: Open Foundation and Fine-Tuned Chat Models\n\nLlama 2: one of the best open source models\n\nAndrew Lukyanenko·Follow\n\nPublished inGoPenAI·15 min read·Jul 20, 2023\n\n--\n\nListen\n\nShare\n\nProject link\n\nModel link\n\nPaper link\n\nThe authors of the work present Llama 2, an assortment of pretrained and fine-tuned large language models (LLMs) with sizes varying from 7 billion to 70 billion parameters. The fine-tuned versions, named Llama 2-Chat, are specifically designed for dialogue applications. These models surpass the performance of existing open-source chat models on most benchmarks, and according to human evaluations for usefulness and safety, they could potentially replace closed-source models. The authors also detail their approach to fine-tuning and safety enhancements for Llama 2-Chat to support the community in further developing and respon

### Split the Text into Chunks

In [7]:
text_splitter=CharacterTextSplitter(separator='\n',
                                    chunk_size=1000,
                                    chunk_overlap=200)

In [8]:
text_chunks=text_splitter.split_documents(data)

### Download the OpenAI Embeddings or Hugging Face Embeddings

In [10]:
embeddings=HuggingFaceEmbeddings()

In [11]:
query_result = embeddings.embed_query("Hello world")
len(query_result)

768

### Convert the Text Chunks into Embeddings and Create a Knowledge Base

In [12]:
vectorstore=FAISS.from_documents(text_chunks, embeddings)

### Create a Large Language Model (LLM) Wrapper

In [ ]:
# llm=ChatOpenAI()

In [ ]:
# llm

In [13]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                          use_auth_token=True,)


model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                              load_in_8bit=True,
                                              #load_in_4bit=True
                                             )

/home/shihab/learning_projects/custom-chat-bot-using-LLM/chatbotenv/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:732: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [ ]:

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [ ]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

In [ ]:

llm.predict("Please provide a concise summary of the Book Alchemist")